In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec 18 07:47:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    57W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os

# if not os.path.exists('/content/yolov5'):
#   %%shell
!git clone https://github.com/ashraffsmd/yolov5-dataverse.git
%cd /content/yolov5-dataverse
!pip install -r requirements.txt

Cloning into 'yolov5-dataverse'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 26 (delta 5), reused 19 (delta 2), pack-reused 0
Unpacking objects: 100% (26/26), done.
/content/yolov5-dataverse
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 1.6 MB/s 
     |████████████████████████████████| 950 kB 17.3 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 1.6 MB 80.8 MB/s 
     |████████████████████████████████| 88 kB 10.0 MB/s 
     |████████████████████████████████| 182 kB 81.8 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 132 kB 82.2 MB/s 
     |████████████████████████████████| 10.3 MB 85.9 MB/s 
     |████████████████████████████████| 79 kB 11.3 MB/s 
     |████████████████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
from zipfile import ZipFile

datasets_dir = '/content'
!cp /content/drive/MyDrive/Dataverse/dataverse.zip /content/
zip_file = datasets_dir + '/dataverse.zip'
ZipFile(zip_file).extractall(path=datasets_dir)
os.remove(zip_file)

In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         167G   24G  144G  15% /
tmpfs            64M     0   64M   0% /dev
shm              41G     0   41G   0% /dev/shm
/dev/root       2.0G  1.1G  910M  54% /sbin/docker-init
tmpfs            42G   40K   42G   1% /var/colab
/dev/sda1       174G   25G  149G  15% /opt/bin/.nvidia
tmpfs            42G     0   42G   0% /proc/acpi
tmpfs            42G     0   42G   0% /proc/scsi
tmpfs            42G     0   42G   0% /sys/firmware
drive            15G   12G  3.4G  78% /content/drive


In [ ]:
%cd /content/yolov5-dataverse

/content/yolov5-dataverse


In [ ]:
import os
import sys
import shutil
from Format import YOLO
import pandas as pd
import numpy as np
from PIL import Image

In [ ]:
def get_data(csv_path, img_path):
    try:
        df = pd.read_csv(csv_path)
        data = {}
        label_counter = 0
        for idx, row in df.iterrows():
            filename = row[0].replace('.jpg', '')
            if row[0] not in data:
                label_counter = 0
                img = Image.open(os.path.join(img_path, filename + '.jpg'))
                img_width = str(img.size[0])
                img_height = str(img.size[1])
                img_depth = 3

                size = {
                    "width": img_width,
                    "height": img_height,
                    "depth": img_depth
                }
                data[filename] = {
                    "size": size,
                    "objects": {}
                }
            bndbox = {
                "xmin": row[2],
                "xmax": row[3],
                "ymin": row[4],
                "ymax": row[5]
            }

            cls = row[1]

            obj_info = {
                "name": cls,
                "bndbox": bndbox
            }

            obj = {
                "num_obj": label_counter + 1,
                str(label_counter): obj_info
            }

            data[filename]["objects"] = {**data[filename]["objects"], **obj}

            label_counter = label_counter + 1
        return True, data
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]

        msg = "ERROR : {}, moreInfo : {}\t{}\t{}".format(e, exc_type, fname, exc_tb.tb_lineno)

        return False, msg

In [ ]:
def prepare_val_dataset(src_csv_path, src_img_path, output_csv_path, output_img_path):
    if os.path.exists(output_img_path):
      shutil.rmtree(output_img_path)
    os.makedirs(output_img_path)
    if os.path.isfile(output_csv_path):
        os.remove(output_csv_path)
    df = pd.read_csv(src_csv_path)
    img_names = df['ImageID'].unique()
    random_imgs = np.random.choice(img_names, 100, False)
    flipped_df = df.loc[df['ImageID'].isin(random_imgs)].reset_index(drop=True)
    for filename in random_imgs:
        img = Image.open(os.path.join(src_img_path, filename))
        rotated = img.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
        rotated.save(os.path.join(output_img_path, filename))
        flipped_df.loc[flipped_df['ImageID'] == filename, 'Width'] = img.size[0]
    flipped_df['Width'] = flipped_df['Width'].astype(int)
    a, b = (flipped_df['Width'] - flipped_df['XMax']), (flipped_df['Width'] - flipped_df['XMin'])
    flipped_df['XMin'] = a
    flipped_df['XMax'] = b
    flipped_df['XMin'] = flipped_df['XMin'].astype(str)
    flipped_df['XMax'] = flipped_df['XMax'].astype(str)
    flipped_df.to_csv(output_csv_path, index=False)

In [ ]:
def main(config):

    if config["datasets"] == "Dataverse":
        yolo = YOLO(os.path.abspath(config["cls_list"]))

        flag, data = get_data(config["label"], config["img_path"])
        if flag == True:
            flag, data = yolo.generate(data)

            if flag == True:
                flag, data = yolo.save(data, config["output_path"], config["img_path"],
                                        config["img_type"], config["manifest_path"])

                if flag == False:
                    print("Saving Result : {}, msg : {}".format(flag, data))

            else:
                print("YOLO Generating Result : {}, msg : {}".format(flag, data))

        else:
            print("BDD Parsing Result : {}, msg : {}".format(flag, data))

    else:
        print("Unkwon Datasets")

def reload_validation_data():
  config1 = {
      "datasets": "Dataverse",
      "img_path": "../datasets/images/train",
      "label": "../datasets/labels.csv",
      "img_type": ".jpg",
      "manifest_path": "./",
      "output_path": "../datasets/labels/train/",
      "cls_list": "./data/dataverse.names",
  }
  config2 = {
      "datasets": "Dataverse",
      "img_path": "../datasets/images/val",
      "label": "../datasets/labels_val.csv",
      "img_type": ".jpg",
      "manifest_path": "./",
      "output_path": "../datasets/labels/val/",
      "cls_list": "./data/dataverse.names",
  }

  # main(config1)
  prepare_val_dataset('../datasets/labels.csv', '../datasets/images/train', '../datasets/labels_val.csv', '../datasets/images/val')
  main(config2)

In [ ]:
from yolov5 import train

In [ ]:
!cp /content/drive/MyDrive/Dataverse/dataverse.yaml ./
!cp /content/drive/MyDrive/Dataverse/yolov5x6.yaml ./

In [ ]:
os.makedirs('./weights')

In [ ]:
for i in range(2, 12):
  src = '/content/drive/MyDrive/Dataverse/weights' + str(i)
  !cp -R $src/* ./weights/
  reload_validation_data()
  train.run(img=1280, batch=30, epochs=300, data='./dataverse.yaml', cfg='./yolov5x6.yaml', weights='./weights/weights/best.pt', name='yolov5l_dataverse', device=0, existOk=True)
  dest = '/content/drive/MyDrive/Dataverse/weights' + str(i+1)
  os.makedirs(dest)
  !cp -R ./runs/train/yolov5l_dataverse/* $dest/

train: weights=./weights/weights/best.pt, cfg=./yolov5x6.yaml, data=./dataverse.yaml, hyp=../../usr/local/lib/python3.8/dist-packages/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5l_dataverse, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, mmdet_tags=False, entity=None, bbox_interval=-1, artifact_alias=latest, neptune_token=None, neptune_project=None, s3_upload_dir=None, upload_dataset=False, img=1280, batch=30, existOk=True



YOLO Generating:|████████████████████████████████████████| 100.0% (100/100)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (100/100)  Complete



YOLOv5 🚀 2022-12-18 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0


  0%|          | 0.00/755k [00:00<?, ?B/s]

Neptune AI: run 'pip install neptune-client' to automatically track and visualize YOLOv5 🚀 runs
TensorBoard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      8800  yolov5.models.common.Conv               [3, 80, 6, 2, 2]              
  1                -1  1    115520  yolov5.models.common.Conv               [80, 160, 3, 2]               
  2                -1  4    309120  yolov5.models.common.C3                 [160, 160, 4]                 
  3                -1  1    461440  yolov5.models.common.Conv               [160, 320, 3, 2]              
  4                -1  8   2259200  yolov5.models.common.C3                 [320, 320, 8]                 
  5                -1  1   1844480  yolov5.models.common.Conv               [320, 640, 3, 2]              
  6                -1 12  

...

Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      8800  yolov5.models.common.Conv               [3, 80, 6, 2, 2]              
  1                -1  1    115520  yolov5.models.common.Conv               [80, 160, 3, 2]               
  2                -1  4    309120  yolov5.models.common.C3                 [160, 160, 4]                 
  3                -1  1    461440  yolov5.models.common.Conv               [160, 320, 3, 2]              
  4                -1  8   2259200  yolov5.models.common.C3                 [320, 320, 8]                 
  5                -1  1   1844480  yolov5.models.common.Conv               [320, 640, 3, 2]              
  6                -1 12  13125120  yolov5.models.common.C3                 [640, 640, 12]                
  7                -1  1   5531520  yolov5.models.common.Conv               [640, 960, 3, 2]             


YOLO Generating:|████████████████████████████████████████| 100.0% (100/100)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (100/100)  Complete



Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      8800  yolov5.models.common.Conv               [3, 80, 6, 2, 2]              
  1                -1  1    115520  yolov5.models.common.Conv               [80, 160, 3, 2]               
  2                -1  4    309120  yolov5.models.common.C3                 [160, 160, 4]                 
  3                -1  1    461440  yolov5.models.common.Conv               [160, 320, 3, 2]              
  4                -1  8   2259200  yolov5.models.common.C3                 [320, 320, 8]                 
  5                -1  1   1844480  yolov5.models.common.Conv               [320, 640, 3, 2]              
  6                -1 12  13125120  yolov5.models.common.C3                 [640, 640, 12]                
  7                -1  1   5531520  yolov5.models.common.Conv               [640, 960, 3, 2]             


YOLO Generating:|████████████████████████████████████████| 100.0% (100/100)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (100/100)  Complete



Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      8800  yolov5.models.common.Conv               [3, 80, 6, 2, 2]              
  1                -1  1    115520  yolov5.models.common.Conv               [80, 160, 3, 2]               
  2                -1  4    309120  yolov5.models.common.C3                 [160, 160, 4]                 
  3                -1  1    461440  yolov5.models.common.Conv               [160, 320, 3, 2]              
  4                -1  8   2259200  yolov5.models.common.C3                 [320, 320, 8]                 
  5                -1  1   1844480  yolov5.models.common.Conv               [320, 640, 3, 2]              
  6                -1 12  13125120  yolov5.models.common.C3                 [640, 640, 12]                
  7                -1  1   5531520  yolov5.models.common.Conv               [640, 960, 3, 2]             